In [1]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [37]:
import random

In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize

EXAMPLE_TEXT = "Hello Mr. Smith, how are you doing today? The weather is great, and Python is awesome. The sky is pinkish-blue. You shouldn't eat cardboard."

print(sent_tokenize(EXAMPLE_TEXT))

['Hello Mr. Smith, how are you doing today?', 'The weather is great, and Python is awesome.', 'The sky is pinkish-blue.', "You shouldn't eat cardboard."]


In [4]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [5]:
example_sent = "This is a sample sentence, showing off the stop words filtration."

stop_words = set(stopwords.words('english'))

word_tokens = word_tokenize(example_sent)

filtered_sentence = [w for w in word_tokens if not w in stop_words]

filtered_sentence = []

for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)

print(word_tokens)
print(filtered_sentence)

['This', 'is', 'a', 'sample', 'sentence', ',', 'showing', 'off', 'the', 'stop', 'words', 'filtration', '.']
['This', 'sample', 'sentence', ',', 'showing', 'stop', 'words', 'filtration', '.']


In [6]:
from nltk.stem import PorterStemmer

In [7]:
ps = PorterStemmer()

In [8]:
new_text = "It is important to by very pythonly while you are pythoning with python. All pythoners have pythoned poorly at least once."

In [9]:
words = word_tokenize(new_text)

for w in words:
    print(ps.stem(w))

It
is
import
to
by
veri
pythonli
while
you
are
python
with
python
.
all
python
have
python
poorli
at
least
onc
.


In [11]:
import gensim

In [ ]:
from gensim import corpora

In [ ]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()

In [27]:
# thanks to https://stackoverflow.com/questions/4182603/how-to-convert-a-string-to-utf-8-in-python
def make_unicode(input):
    if type(input) != unicode:
        input =  input.decode('utf-8')
        return input
    else:
        return input

In [28]:
def tokenize(text):
    lda_tokens = []
    tokens = parser(make_unicode(text))
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [32]:
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /home/anna/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [33]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/anna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [35]:
text_data_orig = ["It is important to by very pythonly while you are pythoning with python. All pythoners have pythoned poorly at least once.",
            "First, we are creating a dictionary from the data, then convert to bag-of-words corpus and save the dictionary and corpus for future use."]

In [44]:
text_data = []
for line in text_data_orig:
    tokens = prepare_text_for_lda(line)
    print tokens
    text_data.append(tokens)

[u'important', u'pythonly', u'pythoning', u'python', u'pythoners', u'pythoned', u'poorly', u'least']
[u'first', u'create', u'dictionary', u'convert', u'words', u'corpus', u'dictionary', u'corpus', u'future']


In [45]:
text_data

[[u'important',
  u'pythonly',
  u'pythoning',
  u'python',
  u'pythoners',
  u'pythoned',
  u'poorly',
  u'least'],
 [u'first',
  u'create',
  u'dictionary',
  u'convert',
  u'words',
  u'corpus',
  u'dictionary',
  u'corpus',
  u'future']]

In [46]:
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

In [47]:
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)

In [48]:
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, u'0.183*"corpus" + 0.183*"dictionary" + 0.100*"words" + 0.100*"future"')
(1, u'0.067*"least" + 0.067*"python" + 0.067*"poorly" + 0.067*"pythoning"')
(2, u'0.109*"pythonly" + 0.109*"important" + 0.109*"pythoned" + 0.109*"python"')
(3, u'0.067*"pythoning" + 0.067*"pythoned" + 0.067*"poorly" + 0.067*"python"')
(4, u'0.067*"poorly" + 0.067*"pythoning" + 0.067*"least" + 0.067*"pythoners"')


In [49]:
import xgboost as xgb